### Data Processing ###

In [39]:
import json
import os

def coco_to_yolo(coco_annotation_file, images_folder, output_folder):
    """
    Convert COCO annotations to YOLO format.

    Args:
    - coco_annotation_file (str): Path to COCO annotations file (JSON)
    - images_folder (str): Path to the folder containing images
    - output_folder (str): Path to output the converted labels and images
    """
    # Load the COCO annotations file
    with open(coco_annotation_file, 'r') as f:
        coco_data = json.load(f)

    # Create output directories for images and labels
    os.makedirs(os.path.join(output_folder, 'images'), exist_ok=True)
    os.makedirs(os.path.join(output_folder, 'labels'), exist_ok=True)

    # Map COCO category IDs to class names
    category_mapping = {category['id']: category['name'] for category in coco_data['categories']}
    class_names = list(category_mapping.values())

    # Process each image in the COCO annotations
    for image_info in coco_data['images']:
        image_id = image_info['id']
        image_filename = image_info['file_name']
        image_path = os.path.join(images_folder, image_filename)

        # Copy image to the new directory
        shutil.copy(image_path, os.path.join(output_folder, 'images', image_filename))

        # Create a label file for each image
        label_file = os.path.join(output_folder, 'labels', f"{os.path.splitext(image_filename)[0]}.txt")
        with open(label_file, 'w') as label_f:
            # Find the annotations for the current image
            annotations = [ann for ann in coco_data['annotations'] if ann['image_id'] == image_id]
            for ann in annotations:
                category_id = ann['category_id']
                class_name = category_mapping[category_id]
                class_id = class_names.index(class_name)  # Get class ID

                # Get bounding box and convert to YOLO format
                bbox = ann['bbox']
                x_min, y_min, width, height = bbox
                x_center = (x_min + width / 2) / image_info['width']
                y_center = (y_min + height / 2) / image_info['height']
                width = width / image_info['width']
                height = height / image_info['height']

                # Write the class ID and normalized bbox to the label file
                label_f.write(f"{class_id} {x_center} {y_center} {width} {height}\n")

    print(f"Conversion complete! Images and labels saved to {output_folder}")

# Set the paths to your files and folders
coco_annotation_file = './instances_default.json'  # Path to COCO annotations JSON file
images_folder = './images'  # Path to the folder containing your images
output_folder = './yolo_dataset'  # Path to save converted YOLO labels and images

# Convert COCO to YOLO format
coco_to_yolo(coco_annotation_file, images_folder, output_folder)


Conversion complete! Images and labels saved to ./yolo_dataset


In [43]:
import random
import shutil

def split_dataset(image_folder, label_folder, train_folder, val_folder, val_ratio=0.2):
    """
    Split the dataset into train and validation sets.

    Args:
    - image_folder (str): Folder containing the images
    - label_folder (str): Folder containing the label files
    - train_folder (str): Folder to store the training images and labels
    - val_folder (str): Folder to store the validation images and labels
    - val_ratio (float): Proportion of the data to use for validation (default is 0.2)
    """
    # Get all image files
    images = [f for f in os.listdir(image_folder) if f.endswith('.jpeg') or f.endswith('.png')]

    # Shuffle the images to randomize the split
    random.shuffle(images)

    # Split the dataset
    val_size = int(len(images) * val_ratio)
    val_images = images[:val_size]
    train_images = images[val_size:]

    # Create train and val directories
    os.makedirs(train_folder, exist_ok=True)
    os.makedirs(val_folder, exist_ok=True)
    os.makedirs(os.path.join(train_folder, 'images'), exist_ok=True)
    os.makedirs(os.path.join(train_folder, 'labels'), exist_ok=True)
    os.makedirs(os.path.join(val_folder, 'images'), exist_ok=True)
    os.makedirs(os.path.join(val_folder, 'labels'), exist_ok=True)

    # Move the images and labels to the appropriate folder
    for image in train_images:
        shutil.copy(os.path.join(image_folder, image), os.path.join(train_folder, 'images', image))
        label_file = image.replace('.jpeg', '.txt').replace('.png', '.txt')
        shutil.copy(os.path.join(label_folder, label_file), os.path.join(train_folder, 'labels', label_file))

    for image in val_images:
        shutil.copy(os.path.join(image_folder, image), os.path.join(val_folder, 'images', image))
        label_file = image.replace('.jpeg', '.txt').replace('.png', '.txt')
        shutil.copy(os.path.join(label_folder, label_file), os.path.join(val_folder, 'labels', label_file))

    print(f"Dataset split complete. {len(train_images)} images for training, {len(val_images)} for validation.")

# Set the paths to your folders
image_folder = './yolo_dataset/images'  # Folder containing images
label_folder = './yolo_dataset/labels'  # Folder containing labels
train_folder = './yolo_dataset/train'  # Folder to store training images/labels
val_folder = './yolo_dataset/val'      # Folder to store validation images/labels

# Split the dataset
split_dataset(image_folder, label_folder, train_folder, val_folder)


Dataset split complete. 87 images for training, 21 for validation.


### Model Training ###


In [53]:
from ultralytics import YOLO

# Load the YOLOv8 segmentation model (pretrained)
model = YOLO("yolov8n.pt")

# Train model
model.train(
    data="data.yaml",
    epochs=50,
    imgsz=640,
    batch=8,
    device="cpu"
)

100%|██████████| 6.25M/6.25M [00:00<00:00, 28.4MB/s]


Ultralytics 8.3.75 🚀 Python-3.12.9 torch-2.6.0 CPU (Apple M2 Pro)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=data.yaml, epochs=50, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None, format=torc

train: Scanning /Users/sonny/Desktop/Code/STIK_GUI/stik_ai/yolo_dataset/train/labels.cache... 87 images, 0 backgrounds, 0 corrupt: 100%|██████████| 87/87 [00:00<?, ?it/s]
val: Scanning /Users/sonny/Desktop/Code/STIK_GUI/stik_ai/yolo_dataset/val/labels.cache... 21 images, 0 backgrounds, 0 corrupt: 100%|██████████| 21/21 [00:00<?, ?it/s]

Plotting labels to runs/detect/train/labels.jpg... 


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs/detect/train
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all         21         21      0.998          1      0.995      0.942



50 epochs completed in 0.276 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.75 🚀 Python-3.12.9 torch-2.6.0 CPU (Apple M2 Pro)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:02<00:00,  1.36s/it]


                   all         21         21      0.997          1      0.995      0.948
Speed: 0.4ms preprocess, 121.8ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x29e58b710>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048,    

In [27]:
metrics = model.val()
print(metrics)

Ultralytics 8.3.75 🚀 Python-3.12.9 torch-2.6.0 CPU (Apple M2 Pro)
YOLOv8n-seg summary (fused): 195 layers, 3,404,320 parameters, 0 gradients, 12.6 GFLOPs

Dataset 'coco.yaml' images not found ⚠️, missing path '/Users/sonny/Desktop/Code/STIK_GUI/stik_ai/datasets/coco/val2017.txt'


100%|██████████| 169M/169M [00:06<00:00, 28.4MB/s] 
Unzipping /Users/sonny/Desktop/Code/STIK_GUI/stik_ai/datasets/coco2017labels-segments.zip to /Users/sonny/Desktop/Code/STIK_GUI/stik_ai/datasets/coco...:  76%|███████▌  | 93191/122232 [00:09<00:02, 9915.83file/s] 


KeyboardInterrupt: 

In [ ]:
results = model("path/to/test_image.jpg")
results.show()


In [ ]:
import os

input_folder = "path/to/new_images"
output_folder = "path/to/annotated_images"
os.makedirs(output_folder, exist_ok=True)

# Run inference on all images in the folder
results = model([os.path.join(input_folder, img) for img in os.listdir(input_folder)])

# Save annotated images
for i, r in enumerate(results):
    r.save(os.path.join(output_folder, f"annotated_{i}.jpg"))  # Saves output images

areas = []

# Iterate through results and calculate area for segmentation masks
for r in results:
    if r.masks:  # Check if masks exist
        for mask in r.masks:
            area = mask.area()  # Get the area of the segmentation mask
            areas.append(area)

print(areas)  # Print the array of areas
